# template から HTML と JS を生成する


In [1]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/TSUMUGI/notebooks/notebools-web
/mnt/e/TSUMUGI


# 古いファイルの消去


In [ ]:
%%bash

rm -rf TSUMUGI/data
rm -rf TSUMUGI/network/phenotype
rm -rf TSUMUGI/network/genesymbol
rm -rf TSUMUGI/network/data

mkdir -p TSUMUGI/data
mkdir -p TSUMUGI/network/phenotype
mkdir -p TSUMUGI/network/genesymbol
mkdir -p TSUMUGI/network/data

echo "*" > TSUMUGI/data/.gitignore
echo "*" > TSUMUGI/network/phenotype/.gitignore
echo "*" > TSUMUGI/network/genesymbol/.gitignore
echo "*" > TSUMUGI/network/data/.gitignore


## index.html の描画に必要なファイルを生成する


In [14]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    if not Path("data", "network", "mp_term_name", f"{mp_term}.json").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    mp_terms[mp_term_name_space] = mp_term

In [15]:
print(list(mp_terms)[:3])
print(len(mp_terms))

['abnormal abdominal wall morphology', 'abnormal adrenal gland morphology', 'abnormal allantois morphology']
549


In [16]:
import json

json.dump(mp_terms, open("data/available_mp_terms.json", "w"), indent=2)

In [17]:
%%bash

cp data/available_mp_terms.json TSUMUGI/data/available_mp_terms.json
cp data/available_gene_symbols.txt TSUMUGI/data/available_gene_symbols.txt
cp data/marker_symbol_accession_id.json TSUMUGI/network/data/marker_symbol_accession_id.json


## index.html を生成


In [12]:
%%bash
grep -v "REMOVE_THIS_LINE" TSUMUGI/template_index.html > TSUMUGI/index.html

# JSON のコピー


In [8]:
%%bash

rm -rf TSUMUGI/network/phenotype/data
rm -rf TSUMUGI/network/genesymbol/data

mkdir -p TSUMUGI/network/phenotype/data
mkdir -p TSUMUGI/network/genesymbol/data

cp -r data/network/mp_term_name/* TSUMUGI/network/phenotype/data
cp -r data/network/gene_symbol/* TSUMUGI/network/genesymbol/data

# 1 min 18 sec

# Template から HTML と JS を生成


In [9]:
%%bash

########################################
# Phenotype
########################################
mkdir -p TSUMUGI/network/phenotype/js

find data/mp_term_name -type f |
sed "s|data/mp_term_name/||" |
sed "s|.csv$||" |
while read mp_term_name_underscore; do
    mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
    impc_url=$(awk -F "\t" -v mp="$mp_term_name_space" '$1 == mp {print $2}' data/mp_term_name_id_url.tsv)
    escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')
    # HTML
    cat TSUMUGI/network/template/template_network_phenotype.html |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_impc_url|${escaped_url}|g" |
    sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
    sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/network/phenotype/"$mp_term_name_underscore".html

    # Javascript
    cat TSUMUGI/network/template/template_network_phenotype.js |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/network/phenotype/js/"$mp_term_name_underscore".js
done
# 24.4s

In [12]:
%%bash

########################################
# Gene Symbol
########################################
mkdir -p TSUMUGI/network/genesymbol/js

cat data/available_gene_symbols.txt |
while read gene_symbol; do
    impc_url="https://www.mousephenotype.org/data/genes/"$(awk -F "\t" -v key="$gene_symbol" '$1 == key {print $2}' data/marker_symbol_accession_id.tsv)
    escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')
    # HTML
    cat TSUMUGI/network/template/template_network_genesymbol.html |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_impc_url|${escaped_url}|g" |
    sed "s|XXX_genesymbol|${gene_symbol}|g" > TSUMUGI/network/genesymbol/"$gene_symbol".html

    # Javascript
    cat TSUMUGI/network/template/template_network_genesymbol.js |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_genesymbol|${gene_symbol}|g" > TSUMUGI/network/genesymbol/js/"$gene_symbol".js
done

# 5 min

## テスト用のデータを test-tsumugi にコピー


In [13]:
%%bash

rm -rf test-tsumugi
mkdir -p test-tsumugi/network/
echo "*" > test-tsumugi/.gitignore

mkdir -p test-tsumugi/data
mkdir -p test-tsumugi/network/phenotype
mkdir -p test-tsumugi/network/genesymbol
(cd test-tsumugi/network/phenotype && mkdir -p data js)
(cd test-tsumugi/network/genesymbol && mkdir -p data js)

In [14]:
%%bash

cp -r TSUMUGI/index.html test-tsumugi/
cp -r TSUMUGI/css test-tsumugi/
cp -r TSUMUGI/js test-tsumugi/
cp -r TSUMUGI/data test-tsumugi/
cp -r TSUMUGI/image test-tsumugi/
cp -r TSUMUGI/network/css test-tsumugi/network/
cp -r TSUMUGI/network/data test-tsumugi/network

In [15]:
%%bash
# Phenotype: male_infertility
find TSUMUGI/network/phenotype/ -type f |
    grep /male_infertility |
    while read path_input; do
        path_output=$(echo $path_input | sed "s|TSUMUGI/|test-tsumugi/|")
        echo $path_output
        cp $path_input $path_output
    done

test-tsumugi/network/phenotype/data/male_infertility.json


test-tsumugi/network/phenotype/js/male_infertility.js
test-tsumugi/network/phenotype/male_infertility.html


In [16]:
%%bash
# Gene: Rab10
find TSUMUGI/network/genesymbol// -type f |
    grep /Rab10 |
    while read path_input; do
        path_output=$(echo $path_input | sed "s|TSUMUGI/|test-tsumugi/|")
        echo $path_output
        cp $path_input $path_output
    done

test-tsumugi/network/genesymbol//data/Rab10.json
test-tsumugi/network/genesymbol//js/Rab10.js


test-tsumugi/network/genesymbol//Rab10.html
